## Time Series Forecasting on airline-passengers dataset

In this project we are going to use airline-passengers dataset for time forecasting using 1.Linear Regression and 2.LSTM and also we are predicting Passengers for next 10 days 

Time series analysis is a statistical method to analyse the past data within a given duration of time to forecast the future. It comprises of ordered sequence of data at equally spaced interval.To understand the time series data & the analysis let us consider an example. Consider an example of Airline Passenger data. It has the count of passenger over a period of time.

importing libraries

In [ ]:
import pandas as pd
import numpy as np 
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
# import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# fix random seed for reproducibility

numpy.random.seed(7)

In [ ]:
df = pd.read_csv('../input/air-passengers/AirPassengers.csv')

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data=df)
plt.title("Number of Passengers accoring to Months")

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
dataframe = pd.read_csv('../input/air-passengers/AirPassengers.csv' , usecols=[1], engine='python')
dataset = dataframe.values


In [ ]:
dataset[:5]

In [ ]:
dataset.shape

In [ ]:
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
dataset[:5]

#### Now spliting data into training and testing set

In [ ]:
dataset.shape

In [ ]:
train_size=len(dataset)*0.65
print(train_size)

In [ ]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
train[86:]

In [ ]:
test[:10]

Now before training the data on the LSTM model, we need to prepare the data so that we can fit it on the model, for this task I will define a helper function:

#### Converting array of matrix into a dataset matrix

In [ ]:

def create_dataset(timeseries_data, n_features):
    X= []
    y = []
    for i in range(len(timeseries_data)):
        #find the end of this pattern
        end_ix = i + n_features
        
        #check if we are beyond the sequence
        if end_ix > len(timeseries_data) -1:
            break
            
        #gather input and output of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)    

In [ ]:
n_steps = 10   
X, y = create_dataset(train, n_steps)
testX, testY = create_dataset(test, n_steps)

In [ ]:
X[:3]

In [ ]:
y[:2]

In [ ]:
X.shape

## Linear Regression

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a= dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+ time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)
    

In [ ]:
time_step =20
X_train, y_train = create_dataset(train, time_step)
X_test, y_test = create_dataset(test, time_step)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model_lr = LinearRegression()

In [ ]:
model_lr.fit(X_train, y_train)

#### Predictions and Model Evaluation

In [ ]:
predictions_lr = model_lr.predict(X_test)

In [ ]:
predictions_lr

In [ ]:
y_test

In [ ]:
print("Acuurancy on training set",model_lr.score(X_train, y_train))

In [ ]:
print("Acuraccy on Testing set",model_lr.score(X_test, y_test))

In [ ]:
predictions_lr.shape, y_test.shape

In [ ]:
pred_df=pd.DataFrame(predictions_lr, y_test).reset_index()

In [ ]:
pred_df_new = pred_df.rename(columns={'index': 'Predictions', 0: 'True_values'})

In [ ]:
pred_df_new.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data= pred_df_new)
plt.title("Predictions VS True Values on Testng set")

## 2.LSTM 

#### Reshape from [samples, timesteps] into [samples, timesteps, features]

In [ ]:
n_features = 1
X= X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
X.shape

### Building LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(50, activation = 'relu', return_sequences = True, input_shape = (n_steps, n_features)))
model.add(LSTM(50, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer= 'adam', loss= 'mse')

Fitting the model 

In [ ]:
model.fit(X, y, epochs = 300, verbose = 1)

In [ ]:
predictions=model.predict(X)

In [ ]:
predictions.shape

In [ ]:
y.shape

In [ ]:
print("Expected Value:",y[10][0])
print('Predicted Value:',predictions[10][0])

#### Predictions for test Data

In [ ]:
n_features = 1
testX= testX.reshape((testX.shape[0], testX.shape[1], n_features))

In [ ]:
test_predictions=model.predict(testX)

In [ ]:
test.shape

we are using last 20 days input for 1 day 

In [ ]:
test.shape
x_input= test[38:]

In [ ]:
# x_input=testX[38:]

In [ ]:
x_input.shape

In [ ]:
x_input = x_input.reshape(1, -1)

In [ ]:
x_input.shape

In [ ]:
temp_input= list(x_input)
temp_input = temp_input[0].tolist()

In [ ]:
len(temp_input)

In [ ]:
x_input = x_input.reshape((1, n_steps,1))
model.predict(x_input)

In [ ]:
# demonstrate prediction for next 10 days
from numpy import array

lst_output=[]
n_steps=10
i=0
while(i<10):
    
    if(len(temp_input)>10):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
#         print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new= np.arange(1,49)
day_pred = np.arange(50, 60)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot(day_new,test)
plt.plot(day_pred,lst_output)

In [ ]:
test[35:]

In [ ]:
lst_output